# Net

> TODO fill in description

In [ ]:
#| default_exp net

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
import asyncio
from collections import deque
from types import MappingProxyType
from typing import Type, Optional, Union, Any, Tuple

import fbdev
from fbdev.utils import StateHandler, StateView, StateCollection, EventHandler, EventCollection, AttrContainer, await_multiple_events, await_any_event, create_task_with_exception_handler
from fbdev.packet import Packet
from fbdev.port import PortType, PortSpec, ConfigPortSpec, PortTypeSpec, PortSpecCollection, BasePort, InputPort, ConfigPort, OutputPort, PortCollection
from fbdev.component import BaseComponent, ComponentFactory, PortSpec, BasePort, InputPort, OutputPort, PortCollection, PortType
from fbdev.graph import EdgeSpec, NodeSpec, Graph, ReadonlyGraph

In [ ]:
#|hide
from fbdev.component import func_component

In [ ]:
#|hide
show_doc(fbdev.net.Edge)

---

### Edge

>      Edge (edge_spec:fbdev.graph.EdgeSpec, parent_process)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class Edge:
    def __init__(self,
                 edge_spec:EdgeSpec,
                 parent_process):
        self._parent_process = parent_process
        self._edge_spec = edge_spec
                
        self._states = StateCollection()
        self._states._add_state(StateHandler("enabled", True), readonly=False)
        self._states._add_state(StateHandler("empty", True), readonly=True)
        self._states._add_state(StateHandler("full", False), readonly=True)
        
        self._events = EventCollection()
        self._events._add_event(EventHandler("has_changed"))
        
        self._maxsize = edge_spec.maxsize
        self._packets = deque(maxlen=edge_spec.maxsize)

    @property
    def parent_process(self): return self._parent_process
    @property
    def edge_spec(self): return self._edge_spec    
    @property
    def states(self): return self._states
    @property
    def events(self): return self._events
    
    @property
    def id(self): return self._edge_spec.id     
    @property
    def tail_connection(self):
        if self.id in self.parent_process.graph.edge_to_tail_connections:
            target_id = self.parent_process.graph.edge_to_tail_connections[self.id]
            if target_id != 0:
                return self.parent_process.nodes[target_id]
            else:
                return self.parent_process
        else:
            return None
    @property
    def head_connection(self):
        if self.id in self.parent_process.graph.edge_to_head_connections:
            target_id = self.parent_process.graph.edge_to_head_connections[self.id]
            if target_id != 0:
                return self.parent_process.nodes[target_id]
            else:
                return self.parent_process
        else:
            return None

    @property
    def maxsize(self): return self.edge_spec.maxsize
        
    def num_packets(self): return len(self._packets)
        
    def _update_status_events(self):        
        if len(self._packets) > 0: self.states._empty.set(False)
        else: self.states._empty.set(True)
        if len(self._packets) == self.maxsize and self.maxsize is not None: self.states._full.set(True)
        else: self.states._full.set(False)
    
    def _load(self, packet:Packet):
        if self.states.full.get(): raise RuntimeError("Port is full.")
        self._packets.append(packet)
        self._update_status_events()
        self.events.has_changed._trigger()
        
    def _unload(self):
        if self.states.empty.get(): raise RuntimeError("Port is empty.")
        packet = self._packets.popleft()
        self._update_status_events()
        self.events.has_changed._trigger()
        return packet
    
    def _load_left(self, packet: Packet):
        if self.states.full.get(): raise RuntimeError("Port is full.")
        self._packets.appendleft(packet)
        self._update_status_events()
        self.events.has_changed._trigger()
    
    def _unload_right(self):
        if self.states.empty.get(): raise RuntimeError("Port is empty.")
        packet = self._packets.pop()
        self._update_status_events()
        self.events.has_changed._trigger()
        return packet
        
    def _unload_all(self):
        if self.states.empty.get(): raise RuntimeError("Port is empty.")
        packets = list(self._packets)
        self._packets.clear()
        self._update_status_events()
        self.events.has_changed._trigger()
        return packets

In [ ]:
#|hide
show_doc(fbdev.net.Node)

---

### Node

>      Node (node_spec:fbdev.graph.NodeSpec, parent_process)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class Node:
    def __init__(self, node_spec:NodeSpec, parent_process):
        if node_spec.component_type.is_factory:
            raise ValueError("Node cannot have a component type that is a ComponentFactory.")
        
        self._parent_process = parent_process
        self._node_spec = node_spec
        self._component_process = None
        
        self._states = StateCollection()
        self._states._add_state(StateHandler("initialised", False), readonly=True)
        self._states._add_state(StateHandler("configured", False), readonly=True)
        self._states._add_state(StateHandler("running", False), readonly=True)
        self._states._add_state(StateHandler("component_executing", False), readonly=True)

        self._events = EventCollection()
        self._events._add_event(EventHandler("stopped_running"))
        self._stop_running_trigger = asyncio.Event()

        self._edge_buses = {}
        self._edge_bus_resting = {}
        self._config_port_handlers = {}
        
    @property
    def parent_process(self): return self._parent_process
    @property
    def node_spec(self): return self._node_spec
    @property
    def id(self): return self.node_spec.id
    @property
    def component_type(self): return self.node_spec.component_type
    @property
    def component_process(self): return self._component_process
    @property
    def port_specs(self): return self.node_spec.component_type.port_specs
    @property
    def ports(self): return self.component_process.ports
    @property
    def states(self): return self._states
    @property
    def events(self): return self._events
    
    @property
    def in_edges(self):
        input_edges = {}
        if self.parent_process is None: return {}
        for port_spec in self.port_specs.iter_ports():
            key = (self.id, port_spec.port_type, port_spec.name)
            if key in self.parent_process.graph.head_connections:
                edge_id = self.parent_process.graph.head_connections[key]
                input_edges[(port_spec.port_type, port_spec.name)] = self.parent_process.edges[edge_id]
        return input_edges
    @property
    def out_edges(self):
        output_edges = {}
        if self.parent_process is None: return {}
        for port_spec in self.port_specs.iter_ports():
            key = (self.id, port_spec.port_type, port_spec.name)
            if key in self.parent_process.graph.tail_connections:
                edge_id = self.parent_process.graph.tail_connections[key]
                output_edges[(port_spec.port_type, port_spec.name)] = self.parent_process.edges[edge_id]
        return output_edges
    @property
    def edges(self):
        return {**self.in_edges, **self.out_edges}
    
    def initialise(self):
        self._component_process = self.component_type()
        self.states._initialised.set(True)
        self.states._configured.set(self.component_process.check_configured())
        
    def reset(self):
        self._component_process = None
        self.states._initialised.set(False)
        self.states._configured.set(False)
                
    async def _handle_in_edge_port_connection(self, edge:Edge, port:BasePort, is_resting:asyncio.Event):
        edge_non_empty = edge.states.empty.get_state_event(False)
        while True:
            await await_multiple_events(port._requesting_packet, edge_non_empty)
            is_resting.clear()
            port._load_packet(edge._unload())
            port._received_packet.set()
            is_resting.set()
            await asyncio.sleep(0)
            
    async def _handle_out_edge_port_connection(self, edge:Edge, port:BasePort, is_resting:asyncio.Event):
        edge_non_full = edge.states.full.get_state_event(False)
        while True:
            await await_multiple_events(port._ready_to_unload, edge_non_full)
            is_resting.clear()
            edge._load(port._unload_packet())
            port._package_sent.set()
            is_resting.set()
            await asyncio.sleep(0)
        
    async def _handle_component_config(self, config_port):
        """Continually awaits receiving packets at a config port, and updates the component process's config."""
        while True:
            packet = await config_port._receive()
            await self.states.component_executing.wait(False) # Doing the awaits in this order guarantees that a config wont be updated whilst the component is executing (though it can be received during it).
            await self.component_process.set_config(config_port.name, packet)
            self.states._configured.set(self.component_process.check_configured())
            await asyncio.sleep(0)
        
    def _start_edge_bus(self, port_type:PortType, port_name:str, edge:Edge):
        port = self.component_process.ports[port_type][port_name]
        is_resting = asyncio.Event()
        if port_type.is_input_port:
            edge_bus = create_task_with_exception_handler(self._handle_in_edge_port_connection(edge, port, is_resting))
        else:
            edge_bus = create_task_with_exception_handler(self._handle_out_edge_port_connection(edge, port, is_resting))
        self._edge_buses[(port_type, port_name)] = edge_bus
        self._edge_bus_resting[(port_type, port_name)] = is_resting
        
    def _stop_edge_bus(self, port_type:PortType, port_name:str):
        self._edge_buses[(port_type, port_name)].cancel()
        del self._edge_buses[(port_type, port_name)]
        del self._edge_bus_resting[(port_type, port_name)]
        
    def _start_config_handler(self, port_name:str):
        port = self.component_process.ports[PortType.CONFIG][port_name]
        config_task = create_task_with_exception_handler(self._handle_component_config(port))
        self._config_port_handlers[port_name] = config_task
        
    def _stop_config_handler(self, port_name:str):
        self._config_port_handlers[port_name].cancel()
        
    async def send_input(self, port_type:PortType, port_name:str, packet:Packet):
        if not port_type.is_input_port: raise ValueError(f"Port type {port_type} is not an input port.")
        port = self.component_process.ports[port_type][port_name]
        if (port_type, port_name) in self._edge_buses:
            await await_multiple_events(port._requesting_packet, self._edge_bus_resting[(port_type, port_name)])
        else:
            await port._requesting_packet.wait()
        port._load_packet(packet)
        port._received_packet.set()
        
    async def get_output(self, port_type:PortType, port_name:str) -> Packet:
        if port_type.is_input_port: raise ValueError(f"Port type {port_type} is not an output port.")
        port = self.component_process.ports[port_type][port_name]
        if (port_type, port_name) in self._edge_buses:
            await await_multiple_events(port._ready_to_unload, self._edge_bus_resting[(port_type, port_name)])
        else:
            await port._ready_to_unload.wait()
        packet = port._unload_packet()
        port._package_sent.set()
        return packet
        
    async def run(self):
        if not self.states.initialised.get(): raise RuntimeError("Node is not initialised.")
        self.states._running.set(True)
        self._stop_running_trigger.clear()
        
        # Start edge buses
        for (port_type, port_name), edge in self.edges.items():
            self._start_edge_bus(port_type, port_name, edge)
        
        # Start config handlers
        for port_name in self.port_specs[PortType.CONFIG]:
            self._start_config_handler(port_name)
        
        # Run component
        await self.component_process.start_background_tasks()
        while not self._stop_running_trigger.is_set():
            await await_any_event(
                self.states.configured.get_state_event(True),
                self._stop_running_trigger,
            )
            if not self._stop_running_trigger.is_set() and not self.component_process.is_passive:
                self.states._component_executing.set(True)
                await self.component_process.execute()
                self.states._component_executing.set(False)
                await asyncio.sleep(0)
            else:
                await self._stop_running_trigger.wait()
        await self.component_process.stop_background_tasks()
        
        # Stop edge buses
        for port_type, port_name in self._edge_buses:
            self._stop_edge_bus(port_type, port_name)
        # Stop config handlers
        for port_name in self.port_specs[PortType.CONFIG]:
            self._stop_config_handler[port_name]
        
        self.states._running.set(False)
             
    async def stop(self):
        if not self.states.running.get(): raise RuntimeError("Node is not running.")
        self._stop_running_trigger.set()
        await self.states.running.wait(False)
        
    async def destroy(self):
        await self.stop()
        await self.component_process.destroy()
        self.reset()
        
    def __del__(self):
        loop = asyncio.get_event_loop()
        if loop.is_running(): create_task_with_exception_handler(self.destroy())
        else: loop.run_until_complete(self.destroy())

In [ ]:
#|hide
show_doc(fbdev.net.CompositeComponentFactory)

---

### CompositeComponentFactory

>      CompositeComponentFactory (graph:fbdev.graph.Graph)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class CompositeComponentFactory(ComponentFactory):
    is_passive = True
    
    def __init__(self, graph:Graph):
        super().__init__()
        
        self._graph = graph
        self._read_only_graph = ReadonlyGraph(graph)
        self._edges = {}
        self._nodes = {}
        
        self._in_buses = []
        self._out_buses = []
        self._config_buses = []
        self._node_runs = {}
        
        # Add edges
        for edge_spec in self.graph.edges.values():
            self._edges[edge_spec.id] = Edge(edge_spec, self)
        # Add nodes
        for node_spec in self.graph.nodes.values():
            self._nodes[node_spec.id] = Node(node_spec, self)
            
        # Initialise nodes
        # TODO allow for more custom initialisation procedure
        for node in self.nodes.values():
            node.initialise()
    
    @property
    def graph(self): return self._read_only_graph
    @property
    def edges(self): return MappingProxyType(self._edges)
    @property
    def nodes(self): return MappingProxyType(self._nodes)
    
    @classmethod
    def get_component(cls, graph):
        port_specs = graph.port_specs
        return cls._create_component_class(class_attrs={'port_specs' : port_specs}, init_args=[graph])
    
    async def _handle_graph_in_port(self, graph_port:InputPort, edge:Edge):
        edge_non_full = edge.states.full.get_state_event(False)
        while True:
            packet = await graph_port.receive() # TODO change the packet parent registry
            await edge_non_full.wait()
            edge._load(packet)
            
    async def _handle_graph_out_port(self, graph_port:OutputPort, edge:Edge):
        edge_non_empty = edge.states.empty.get_state_event(False)
        while True:
            await edge_non_empty.wait()
            packet = edge._unload()
            await graph_port.put(packet)
        
    async def _forward_config_into_graph(self, edge:Edge, packet:Packet):
        await edge.states.full.get_state_event(False)
        edge._load(packet)
        
    def set_config(self, name:str, packet:Packet):
        """Overloads BaseComponent.set_config to forward the config to child nodes, where such edges have been defined."""
        if (PortType.CONFIG, name) in self.graph.graph_tail_connections:
            edge_id = self.graph.graph_tail_connections[(PortType.CONFIG, name)]
            task = create_task_with_exception_handler(self._forward_config_into_graph(self.edges[edge_id], packet))
            self._config_buses.append(task)
        else:
            super().set_config(name, packet)
    
    async def execute(self):
        await super().execute()
    
    async def start_background_tasks(self):
        for (port_type, port_name), edge_id in self.graph.graph_tail_connections.items():
            if port_type == PortType.CONFIG:
                continue # Config edges are handled separately
            task = create_task_with_exception_handler(self._handle_graph_in_port(self.ports[port_type][port_name], self.edges[edge_id]))
            self._in_buses.append(task)
            
        for (port_type, port_name), edge_id in self.graph.graph_head_connections.items():
            task = create_task_with_exception_handler(self._handle_graph_out_port(self.ports[port_type][port_name], self.edges[edge_id]))
            self._out_buses.append(task)
        
        for node in self.nodes.values():
            task = create_task_with_exception_handler(node.run())
            self._node_runs[node.id] = task
            
    async def stop_background_tasks(self):
        for bus in self._in_buses+self._out_buses+self._config_buses:
            bus.cancel()
        self._in_buses.clear()
        self._out_buses.clear()
        self._config_buses.clear()
        
        for node_id in self._node_runs:
            await self.nodes[node_id].stop()
    
    async def destroy(self):
        await super().destroy()
        for node in self.nodes.values():
            await node.destroy()


In [ ]:
#|hide
show_doc(fbdev.net.Net)

---

### Net

>      Net (node_spec:fbdev.graph.NodeSpec, parent_process=None)

*A net is a Node that exposes some additional functionality, when its component is a composite component.*

In [ ]:
#|export
class Net(Node):
    """A net is a Node that exposes some additional functionality, when its component is a composite component."""
    def __init__(self, node_spec:NodeSpec, parent_process=None):
        if not issubclass(node_spec.component_type.parent_factory, CompositeComponentFactory):
            raise ValueError("Nets must have a component type created from a CompositeComponentFactory.")
        super().__init__(node_spec, parent_process)
        
    @classmethod
    def from_graph(cls, graph:Graph, parent_process=None):
        component_type = CompositeComponentFactory.get_component(graph)
        node_spec = NodeSpec(component_type)
        return cls(node_spec, parent_process)

In [ ]:
@func_component()
def add_one(a:int) -> int:
    print("In add_one")
    return a+1

@func_component()
def copier(a:int) -> Tuple[int, int]:
    print("In copier")
    return a, a

@func_component()
def printer(a:int) -> None:
    print("In printer1:", a)
    
@func_component()
def sender(a:int):
    print("In sender")
    return a
    
g = Graph(PortSpecCollection(
    input=PortTypeSpec(in1=PortSpec(dtype=int)),
    output=PortTypeSpec(out=PortSpec(dtype=int))
))

g.add_node(NodeSpec(add_one))
g.add_node(NodeSpec(copier))
g.add_node(NodeSpec(printer))
g.add_node(NodeSpec(sender))

g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())

g.connect_edge_to_graph_port(PortType.INPUT, 'in1', 0)
g.connect_edge_to_node('add_one', PortType.INPUT, 'a', 0)

g.connect_edge_to_node('add_one', PortType.OUTPUT, 'out', 1)
g.connect_edge_to_node('copier', PortType.INPUT, 'a', 1)

g.connect_edge_to_node('copier', PortType.OUTPUT, 'out0', 2)
g.connect_edge_to_node('printer', PortType.INPUT, 'a', 2)

g.connect_edge_to_node('copier', PortType.OUTPUT, 'out1', 3)
g.connect_edge_to_node('sender', PortType.INPUT, 'a', 3)

g.connect_edge_to_node('sender', PortType.OUTPUT, 'out', 4)
g.connect_edge_to_graph_port(PortType.OUTPUT, 'out', 4)

net = Net.from_graph(g)
net.initialise()

async def packet_sender():
    print("Awaiting sending initial packet")
    await net.send_input(PortType.INPUT, 'in1', Packet(1))
    print("Sent initial packet")
        
async def packet_receiver():
    packet = await net.get_output(PortType.OUTPUT, 'out')
    packet_payload = await packet.consume()
    print("Final output:", packet_payload)
    
net_task = asyncio.create_task(net.run())
    
await asyncio.gather(
    asyncio.create_task(packet_sender()),
    asyncio.create_task(packet_receiver()),
);

net_task.cancel();

Awaiting sending initial packet
Sent initial packet
In add_one
In copier
In printer1: 2
In sender
Final output: 2
